In [1]:
import random
import json
import  pickle
import numpy as np

import nltk
import tflearn 
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split

from tensorflow.keras.models import  Sequential,load_model
from  tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.optimizers import Adam


Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
import tensorflow as tf
from tensorflow.python.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings("ignore")
tf.__version__

'2.10.0'

In [3]:
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

In [4]:
lemmatizer  = WordNetLemmatizer()
intents = json.loads(open('data.json').read())

In [5]:
words = []
classes = []
documents = []
ignore_letters = ['?','!','.',',']

In [6]:

training = []


In [7]:
def clean_up_sentence(sentence):
    sentence_words =nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words if word not in ignore_letters]
    return sentence_words

In [8]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = clean_up_sentence(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

#words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

In [9]:
def bag_of_words(sentence_words):
    # sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for word in sentence_words:
        for i,w in enumerate(words):
            if w == word:
                bag[i]=1    
    return bag

In [10]:
for document in documents:
    # bag = []
    # word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    # for word in words : 
    #     bag.append(1) if word in word_patterns else bag.append(0)
    bag = bag_of_words(document[0])
    output_row = [0] * len(classes)
    output_row[classes.index(document[1])]=1
    training.append([bag,output_row])
    

In [11]:
random.shuffle(training)
training  = np.array(training)

In [12]:
X_train = list(training[:,0])
y_train = list(training[:,1])

model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dense(len(y_train[0]),activation='sigmoid'))


In [13]:
# tf.compat.v1.reset_default_graph() #Reset Underlying Graph data

# #Building our own Neural Network
# net = tflearn.input_data(shape=[None, len(X_train[0])])
# net = tflearn.fully_connected(net, 10)
# net = tflearn.fully_connected(net, 10)
# net = tflearn.fully_connected(net, len(y_train[0]), activation="softmax")
# net = tflearn.regression(net)

# #Defining Model and setting up tensorboard
# model = tflearn.DNN(net, tensorboard_dir="tflearn_logs") 

# #Now we have setup model, now we need to train that model by fitting data into it by model.fit()
# #n_epoch is the number of times that model will se our data during training
# model.fit(X_train, y_train, n_epoch=1000, batch_size=8, show_metric=True) 
# model.save("model.tflearn") #Saving the model

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               10112     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 11)                715       
                                                                 
Total params: 23,243
Trainable params: 23,243
Non-traina

In [15]:
adam = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',optimizer=adam ,metrics=['accuracy'])

In [16]:
early_stop = EarlyStopping(monitor='loss', patience=2)
hist = model.fit(np.array(X_train),np.array(y_train),epochs=200,batch_size=5,verbose=2)

Train on 50 samples
Epoch 1/200
50/50 - 0s - loss: 2.4595 - acc: 0.0600 - 100ms/epoch - 2ms/sample
Epoch 2/200
50/50 - 0s - loss: 2.4217 - acc: 0.0400 - 9ms/epoch - 183us/sample
Epoch 3/200
50/50 - 0s - loss: 2.3571 - acc: 0.1200 - 12ms/epoch - 242us/sample
Epoch 4/200
50/50 - 0s - loss: 2.3283 - acc: 0.2000 - 11ms/epoch - 224us/sample
Epoch 5/200
50/50 - 0s - loss: 2.2751 - acc: 0.2400 - 13ms/epoch - 251us/sample
Epoch 6/200
50/50 - 0s - loss: 2.3045 - acc: 0.2600 - 11ms/epoch - 210us/sample
Epoch 7/200
50/50 - 0s - loss: 2.2908 - acc: 0.1800 - 11ms/epoch - 221us/sample
Epoch 8/200
50/50 - 0s - loss: 2.2135 - acc: 0.2600 - 11ms/epoch - 220us/sample
Epoch 9/200
50/50 - 0s - loss: 2.2662 - acc: 0.2600 - 12ms/epoch - 242us/sample
Epoch 10/200
50/50 - 0s - loss: 2.1673 - acc: 0.2800 - 15ms/epoch - 306us/sample
Epoch 11/200
50/50 - 0s - loss: 2.1582 - acc: 0.2800 - 15ms/epoch - 290us/sample
Epoch 12/200
50/50 - 0s - loss: 2.0653 - acc: 0.2800 - 15ms/epoch - 292us/sample
Epoch 13/200
50/50 

In [17]:
m
odel.save('model.h5',hist)

NameError: name 'm' is not defined

In [ ]:
igk
model = load_model('model.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [ ]:
def predict_class(sentence):
    bow = bag_of_words(clean_up_sentence(sentence))
    res = model.predict(np.array([np.array(bow)]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r > ERROR_THRESHOLD]
    
    results.sort(key=lambda x: x[1],reverse=True)
    return_list  =[]
    
    for r in results:
        return_list.append({'intent':classes[r[0]], 'probabilty':str(r[1])})
    
    return return_list
        

In [ ]:
def get_reposnse(intents_list,intent_json):
    tag = intents_list[0]['intent']
    list_of_intents = intent_json['intents']
    result =""
    for i in list_of_intents:
        if i['tag'] == tag:
            result =random.choice(i['responses'])
            break
    return result

In [ ]:
print("strat!!!!")

while True:
    message = input("")
    ints = predict_class(message)
    res = get_reposnse(ints,intents)
    print(res)

strat!!!!
HI there , wanna check our new products.
HI there , wanna check our new products.
Hello, thanks for visiting
